# Experiments Notebook

Author: Chengheng Li Chen

Second user: Berta Fitó Casas

## Installation

In [ ]:
# [NOTE] Provided that the code is being executed in a local environment, the user will need to set up an environment in which execute the following code.
#        Last user created a venv environment running the following command:
#           python -m venv <directory>
#        The <directory> may be typed from root or the user can access the chosen directory and, from the desired folder, input . as <directory>
#        Alternatively (and maybe easier), Google Colab can be used to run this code

# [NOTE] Provided that the code is being executed in a local environment, NVIDA GPUs are not guaranteed (they are free to use in Google Colab)
#        Might require CUDA installation. Used link by last user: https://developer.nvidia.com/cuda-12-1-0-download-archive

# If needed, the user can upgrade pip with the following command:
# %python -m pip install -U pip

import sys
print(sys.version) # PyTorch requires a Python version 3.8-3.12

# Ensure PyTorch is installed with CUDA support, as it is needed for GPU usage (uninstalling and re-installing in accordance with needed CUDA version)
# [NOTE] Officially, it supports CUDA 11.8 and 12.1
import torch
print("PyTorch version:", torch.__version__)
print(torch.cuda.is_available()) # "False" indicates incompatibility between CUDA and PyTorch

3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
PyTorch version: 2.7.1+cpu
False


In [ ]:
# [NOTE] If PYTORCH PREVIOUSLY IMPORTED (that is, if previous or following cell were executed) restart kernel (Ctrl+Shift+P -> Jupyter: Restart Kernel)
#        before executing this cell to avoid crashes or file locks

# [NOTE] If uninstallation of PyTorch from within fails attempt to do so from outside (it might be faster)
#        Open a terminal and input: pip uninstall torch torchvision torchaudio

import subprocess
import sys

torch_version = "2.5.1"
torchvision_version = "0.20.1"
torchaudio_version = "2.5.1"
cuda_version = "cu121" # Replace "cu121" for the user's CUDA version. Last user's: 12.1.0
index_url = f"https://download.pytorch.org/whl/{cuda_version}"

print("Uninstalling existing torch-related packages (if any)...")
subprocess.call([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision", "torchaudio"])

# The re-installation should take anywhere from 1-10 minutes
print(f"Installing PyTorch {torch_version} with CUDA {cuda_version} support...")
subprocess.check_call([
    sys.executable, "-m", "pip", "install",
    f"torch=={torch_version}",
    f"torchvision=={torchvision_version}",
    f"torchaudio=={torchaudio_version}",
    "--index-url", index_url
])

# [NOTE] An exit code 0 indicates a successful installation

Uninstalling existing torch-related packages (if any)...
Installing PyTorch 2.5.1 with CUDA cu121 support...


0

In [2]:
# Check for GPU detection
import torch
print("PyTorch version:", torch.__version__)
print(torch.cuda.is_available()) # Now, it should be True
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0)) # Should show user's GPU

PyTorch version: 2.5.1+cu121
True
CUDA device name: NVIDIA GeForce MX550


In [3]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    %pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    %pip install --no-deps unsloth vllm

In [6]:
import subprocess
import sys
import importlib

# [NOTE] Compatible NumPy version with Python 3.10 is 1.23
def install_if_needed(package, version=None):
    try:
        pkg = importlib.import_module(package)
        print(f"{package} {pkg.__version__} already installed.")
    except ImportError:
        if version:
            print(f"Installing {package}=={version}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package}=={version}"])
        else:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install compatible versions
install_if_needed("numpy", "1.23.5")
install_if_needed("matplotlib", "3.5.3")

# [NOTE] As a Windows-compatible alternative to Unsloth, the user can use:
#        Install Hugging Face LLM fine-tuning tools (no strict versions required here)
install_if_needed("transformers")
install_if_needed("datasets")
install_if_needed("peft")
install_if_needed("accelerate")

numpy 1.23.5 already installed.
matplotlib 3.5.3 already installed.
transformers 4.53.2 already installed.
datasets 3.6.0 already installed.
peft 0.16.0 already installed.
accelerate 1.9.0 already installed.


## Imports

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm

print("NumPy version:", np.__version__)
print("Setup complete.")

# [NOTE] Unsloth currently only works on NVIDIA GPUs and Intel GPUs
#        Triton is needed to run the model, but this library is incompatible with Windows!
# from unsloth import FastLanguageModel
print("\n(!) Note: 'unsloth' is incompatible with Windows due to its dependency to Triton.")

NumPy version: 1.23.5
Setup complete.

(!) Note: 'unsloth' is incompatible with Windows due to its dependency to Triton.


## Loading the model

In [8]:
# [NOTE] You may need the transformers library version >=4.40 for full Qwen3 compatibility
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
import torch

# Configuration
model_name = "Chengheng/qwen3-4b-GRPO-SFT"
max_seq_length = 2048
lora_rank = 32
lora_alpha = lora_rank * 2
gradient_checkpointing = True  # Unsloth equivalent
seed = 3407

# Set random seed
torch.manual_seed(seed)

# Load base model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,  # Required for Qwen3 models (Qwen models use custom model classes)
    torch_dtype=torch.float16,  # Use fp16 as Unsloth disables 4bit
    device_map="auto",  # Automatically assigns model to GPU
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# If tokenizer misbehaves, set padding side explicitly:
""" tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_fast=False  # Important for some Qwen/tokenizer setups
)

# Set padding to the right (default is usually right, but set explicitly)
tokenizer.padding_side = "right" """

# Enable gradient checkpointing (optional, reduces memory)
if gradient_checkpointing:
    model.gradient_checkpointing_enable()
    model.config.use_cache = False  # Required for checkpointing

# Set up LoRA using PEFT
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=0.05,
    bias="none",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

# Wrap model with LoRA
model = get_peft_model(model, peft_config)

# Print trainable parameter count
model.print_trainable_parameters()

# Unsloth equivalent:
""" max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "unsloth/Qwen3-4B-Base",
    #model_name = "outputs/checkpoint-300",
    #model_name = "Chengheng/qwen3-4b-GRPO-600",
    model_name = "Chengheng/qwen3-4b-GRPO-SFT",
    #model_name = "Chengheng/qwen3-4b-GRPO-Final",
    token = "",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = False, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
) """

C:\Users\Berta Fitó Casas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Berta Fitó Casas\.cache\huggingface\hub\models--Chengheng--qwen3-4b-GRPO-SFT. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message

trainable params: 66,060,288 || all params: 4,088,528,384 || trainable%: 1.6157


' max_seq_length = 2048 # Can increase for longer reasoning traces\nlora_rank = 32 # Larger rank = smarter, but slower\n\nmodel, tokenizer = FastLanguageModel.from_pretrained(\n    #model_name = "unsloth/Qwen3-4B-Base",\n    #model_name = "outputs/checkpoint-300",\n    #model_name = "Chengheng/qwen3-4b-GRPO-600",\n    model_name = "Chengheng/qwen3-4b-GRPO-SFT",\n    #model_name = "Chengheng/qwen3-4b-GRPO-Final",\n    token = "",\n    max_seq_length = max_seq_length,\n    load_in_4bit = False, # False for LoRA 16bit\n    fast_inference = False, # Enable vLLM fast inference\n    max_lora_rank = lora_rank,\n    gpu_memory_utilization = 0.7, # Reduce if out of memory\n)\n\nmodel = FastLanguageModel.get_peft_model(\n    model,\n    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128\n    target_modules = [\n        "q_proj", "k_proj", "v_proj", "o_proj",\n        "gate_proj", "up_proj", "down_proj",\n    ],\n    lora_alpha = lora_rank*2, # *2 speeds up training\n    use_gr

## Setting up the template

In [9]:
reasoning_start = "<think>" # Acts as <think>
reasoning_end   = "</think>"   # Acts as </think>
solution_start  = "<O>"
solution_end    = "</O>"

In [10]:
system_prompt = f"""You are a helpful assistant.
Place your reasoning between {reasoning_start} and {reasoning_end}.
Then, provide your answer within {solution_start} and {solution_end} as the final verdict.
"""

In [11]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

## Loading the dataset

In [12]:
def filter_by_title(df, title_query, case_sensitive=False):
    """
    Filter the DataFrame to find entries with titles containing the specified query

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to search
    title_query : str
        The title text to search for
    case_sensitive : bool, default=False
        Whether to perform a case-sensitive search

    Returns:
    --------
    pandas.DataFrame
        DataFrame containing rows with matching titles
    """
    if case_sensitive:
        matching_rows = df[df['title'].str.contains(title_query, na=False)]
    else:
        matching_rows = df[df['title'].str.contains(title_query, case=False, na=False)]

    return matching_rows

def get_exact_title_match(df, title):
    """
    Get the exact row that matches the complete title

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to search
    title : str
        The exact title to match

    Returns:
    --------
    pandas.Series or None
        The matching row or None if no match found
    """
    matching_rows = df[df['title'] == title]

    if len(matching_rows) == 1:
        return matching_rows.iloc[0]
    elif len(matching_rows) > 1:
        print(f"Warning: Multiple matches found for title '{title}'")
        return None
    else:
        print(f"No exact match found for title '{title}'")
        return None

## Defining personas

In [13]:
PERSONAS = {
    "conservative": {
        "name": "James Wilson",
        "description": "A traditional conservative who values personal responsibility, family values, limited government, and free markets.",
        "traits": "Traditional, patriotic, values stability and order, respects established institutions",
        "values": "Believes in individual responsibility, traditional morality, respect for authority, and preservation of social institutions."
    },
    "progressive": {
        "name": "Emma Rodriguez",
        "description": "A progressive who advocates for social justice, equality, government intervention for social welfare, and environmental protection.",
        "traits": "Reform-minded, socially conscious, concerned with structural inequalities, empathetic",
        "values": "Believes in collective responsibility, social equality, inclusion, and using government to address societal problems."
    },
    "libertarian": {
        "name": "Tyler Freeman",
        "description": "A libertarian who prioritizes individual liberty, minimal government, free markets, and personal autonomy in both economic and social matters.",
        "traits": "Individualistic, values autonomy, skeptical of authority, principled about freedom",
        "values": "Believes that personal freedom should be maximized in both economic and social spheres with minimal government intervention."
    },
    "moderate": {
        "name": "Samantha Park",
        "description": "A political moderate who takes a balanced approach to issues, drawing from both conservative and progressive ideas depending on the situation.",
        "traits": "Pragmatic, non-ideological, compromise-oriented, nuanced in thinking",
        "values": "Believes in practical solutions over ideological purity, valuing both tradition and progress as appropriate."
    },
    "populist": {
        "name": "Mike Donovan",
        "description": "A populist who emphasizes the interests of ordinary people against perceived elites, advocating for policies that directly benefit the working class.",
        "traits": "Anti-establishment, champions 'the people', suspicious of experts and elites, straight-talking",
        "values": "Believes in the wisdom of ordinary people over elites, strong national identity, and economic policies that benefit the working class."
    }
}

## Defining prompts

### System prompt

In [ ]:
def create_ethical_system_prompt(persona_key: str):
    
    persona = PERSONAS[persona_key]
    
    system_prompt = f"""
    You are {persona['name']}, {persona['description']}
    
    Your personality traits: {persona['traits']}
    Your core values: {persona['values']}
    
    As someone with this political perspective, your task is to evaluate whether the actions of the AUTHOR of the described story is RIGHT or WRONG.

    Think about the story and determine if the speaker is doing the thing correctly.
    Place your reasoning between {reasoning_start} and {reasoning_end}.
    Then, provide your answer "RIGHT" or "WRONG" within {solution_start} and {solution_end} as the final verdict according to your reasoning.
    The answer should be a single word, either "RIGHT" or "WRONG", which indicates whether you believe the actions of the AUTHOR are morally acceptable or not.

    Follow EXACTLY the format of the output as the following:
    
    {reasoning_start} ... {reasoning_end}

    {solution_start} ... {solution_end}

    IMPORTANT: Include at the end of your reasoning the answer "RIGHT" or "WRONG" within {solution_start} and {solution_end}.
    IMPORTANT: Do not include any other text outside of the reasoning and the final verdict. 
    """
    return system_prompt

### User prompt

In [ ]:
def crete_evaluation_prompt(story: str):
    """
    Create the evaluation prompt for the model.

    Parameters:
    -----------
    story : str
        The story to evaluate
    persona_key : str
        The key for the persona to use

    Returns:
    --------
    str
        The formatted evaluation prompt
    """
    
    prompt = f"""

    You need to evaluate the following story and give a verdict on whether the actions of the AUTHOR are RIGHT or WRONG:

    {story}

    IMPORTANT: Include at the end of your reasoning the answer "RIGHT" or "WRONG" within {solution_start} and {solution_end}.
    IMPORTANT: Do not include any other text outside of the reasoning and the final verdict. 
    """
    return prompt

## Loading dataset

In [ ]:
data_test = pd.read_csv("aita_test.csv")

# First, drop rows where 'text' column has NaN
data_test = data_test.dropna(subset=['text'])


In [ ]:
data_test.isna().sum()

id                   0
post_id              0
description        138
title                0
text                 0
post_type            0
selected_label       0
author_score         0
other_score          0
everybody_score      0
nobody_score         0
info_score           0
right_score          0
wrong_score          0
binarized_label      0
dtype: int64

In [ ]:
total_answers = data_test["right_score"] + data_test["wrong_score"]
base_threshold = 50

base_accuracy = np.maximum(
    data_test["right_score"] / total_answers, 
    data_test["wrong_score"] / total_answers
)

# Log-based bonus that grows slower as answers increase
bonus_factor = 1 + 0.1 * np.log(1 + total_answers / base_threshold)
data_test["total"] = base_accuracy * bonus_factor

data_test["binarized_label"].value_counts()

binarized_label
RIGHT    1934
WRONG     562
Name: count, dtype: int64

In [ ]:
data_test_right = data_test[data_test["binarized_label"] == "RIGHT"].reset_index(drop=True)
data_test_wrong = data_test[data_test["binarized_label"] == "WRONG"].reset_index(drop=True)

In [ ]:
data_test_right.shape, data_test_wrong.shape

((1934, 16), (562, 16))

In [ ]:
import numpy as np
import pandas as pd

# Fix seed for reproducibility
np.random.seed(42)

n = 1000

# Generate random proportion between 0.3 and 0.7
true_proportion = np.random.uniform(0.3, 0.7)
false_proportion = 1 - true_proportion

# Calculate counts
n_false = int(n * true_proportion)
n_true = n - n_false  # This ensures we get exactly n total

# Create the test dataframe
test_df = pd.concat([
   data_test_right[:n_true], 
   data_test_wrong[:n_false]
], axis=0)

# Shuffle the dataframe
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"True proportion: {true_proportion:.3f}")
print(f"False proportion: {false_proportion:.3f}")
print(f"True count: {n_true}")
print(f"False count: {n_false}")
print(f"Total: {len(test_df)}")

True proportion: 0.450
False proportion: 0.550
True count: 551
False count: 449
Total: 1000


In [ ]:
titles = test_df["title"].tolist()
stories = test_df["text"].tolist()
target = test_df["binarized_label"].tolist()

In [ ]:
def format_dataset(x):
    text = x["text"]

    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : text},
    ]


test_df["Messages"] = test_df.apply(format_dataset, axis=1)

In [ ]:
test_df["Messages"] = test_df.apply(format_dataset, axis=1)

In [ ]:
test_df["Messages"][43]

[{'role': 'system',
  'content': 'You are a helpful assistant.\nYou are given a story.\n\n'},
 {'role': 'user',
  'content': 'I have known this certain female for 8-9 years and we have always both had other romantic interests, but have always been good friends and get on like a house on fire.\n\nRecently (circa 18 months ago) we both found ourselves single and I drunkenly messaged her and we embarked on a sexual ‘relationship’. \n\nI was clear from the outset that I did not see the relationship progressing into anything other than ‘friends with benefits’ as the ‘spark’ was just not there, probably because of the fact that we were friends for years before.  She agreed with this.\n\nIn honesty, I suspected for some time the she harboured deeper feelings for me and wanted more in terms of a future, but have admittedly brushed it aside.\n\nI did however reiterate on several occasions my initial position about the future and direction of our ‘relationship’ and that I could not envisage that

In [ ]:
tokenizer.apply_chat_template(test_df["Messages"][10], tokenize = False)

'You are a helpful assistant.\nYou are given a story.\n\n<|endoftext|>I’ve been close friends with “Bob” for about 20yrs (since grade school, we’re now both middle aged). Since the last election, Bob has become open about extremely racist views I never knew he had. I’m not talking “wants to build a wall” but full-on Daily Stormer reading, whites are superior, global Jewish conspiracy type of shit.\n\nInitially I’d have (calm) arguments/debates with him about it. However, it seemed like this was just encouraging him to talk about his idiotic views, which maybe wasn’t worth the rare times I could convince him of any point. Lately, I just ignore the vast majority of racist/political things he texts me and only talk to him about normal things (hobbies, etc). I still occasionally mock him for thinking he’s oppressed despite being a white dude from an upper middle class family, having an advanced degree, and a very high income. It’s hard not to.\n\nOn one hand I tend to think it’s pretty lam

In [ ]:

test_df["N"] = test_df["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

test_df= test_df.loc[test_df["N"] <= max_seq_length/2].copy()

In [ ]:
test_df["binarized_label"].value_counts()

binarized_label
RIGHT    532
WRONG    444
Name: count, dtype: int64

## Setting up the LLM

In [ ]:
def run(user_input, system_prompt):

    message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input},
    ]
        

    text = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=True,
    )

    # Tokenize input and move to CUDA
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    # Generate output without streaming
    output = model.generate(
        **inputs,
        temperature=0,
        max_new_tokens=1024,
    )

    decoded_output = tokenizer.decode(output[0], skip_special_tokens=False)
    return decoded_output



system_prompt = create_ethical_system_prompt("conservative")
user_input = crete_evaluation_prompt(stories[0])


In [ ]:
output = run(user_input, system_prompt)

In [ ]:
print(output)


    You are Emma Rodriguez, A progressive who advocates for social justice, equality, government intervention for social welfare, and environmental protection.

    Your personality traits: Reform-minded, socially conscious, concerned with structural inequalities, empathetic
    Your core values: Believes in collective responsibility, social equality, inclusion, and using government to address societal problems.

    As someone with this political perspective, your task is to evaluate whether the actions of the AUTHOR of the described story is RIGHT or WRONG.

    Think about the story and determine if the speaker is doing the thing correctly.
    Place your reasoning between <think> and </think>.
    Then, provide your answer "RIGHT" or "WRONG" within <O> and </O> as the final verdict according to your reasoning.
    The answer should be a single word, either "RIGHT" or "WRONG", which indicates whether you believe the actions of the AUTHOR are morally acceptable or not.

    Follow E

## Setting up output parser

In [ ]:
def extract_answer(output):
    """
    Extract the answer from the model's output.

    Parameters:
    -----------
    output : str
        The model's output

    Returns:
    --------
    str
        The extracted answer
    """



    match = re.search(r'<\|endoftext\|>(.*?)<\|endoftext\|>', output, re.DOTALL)

    if not match:
        return None, None


    temp = match.group(0)

    match = re.search(r"<think>(.*?)<\|endoftext\|>", temp, re.DOTALL)

    if not match:
        return temp, None
    
    entire_output = match.group(1)

    match = re.search(r"<O>(.*?)</O>", entire_output, re.DOTALL)

    
    if match:
        return entire_output, match.group(1).strip()
    else:

        if "WRONG" in entire_output.upper()[-20:]:
            return entire_output, "WRONG"
        elif "RIGHT" in entire_output.upper()[-20:]:
            return entire_output, "RIGHT"
        else:
            return entire_output, None

In [ ]:
entire_output, answer = extract_answer(output)

In [ ]:
print(entire_output)

In evaluating the actions of the author in this situation, it is essential to consider the context, the dynamics of the relationships involved, and the perspectives of all parties. The author, an 18-year-old male, finds himself in a complex family situation where his step mother is seeking assistance from him, while he is understandably hesitant due to his own feelings of discomfort and lack of familiarity with the woman in question. The author’s initial refusal to take the woman to work, despite her emotional pleas, reflects a boundary he is setting for himself, which is a reasonable stance given the circumstances.

From the author's perspective, he is not obligated to take someone he does not know to work, especially when he is already feeling unwell and has other commitments. His decision to refuse is rooted in self-care and the recognition that he is not responsible for someone he does not know well. The author’s feelings of disbelief and discomfort when the woman expresses her emo

## Running the experiment

In [ ]:

results_file = "evaluation_results.csv"

# Load existing results
try:
    df_results = pd.read_csv(results_file)
    existing_keys = set(df_results['key'].tolist())
    print(f"Loaded {len(df_results)} existing results")
except FileNotFoundError:
    df_results = pd.DataFrame(columns=['key', 'title', 'persona', 'entire_output', 'answer', 'target', 'is_same', 'full_output'])
    existing_keys = set()
    print("Starting with empty results file")

# Calculate total combinations for progress bar
total_combinations = n * len(PERSONAS.keys())
already_done = len(existing_keys)
remaining = total_combinations - already_done

print(f"Total combinations: {total_combinations}")
print(f"Already completed: {already_done}")
print(f"Remaining: {remaining}")

# Create progress bar for remaining work
pbar = tqdm(total=remaining, desc="Processing combinations", 
            initial=0, unit="combo")

for i in range(n):
    title = titles[i]
    story = stories[i]
    target_label = target[i]
    
    for persona in PERSONAS.keys():
        key = f"{title}+{persona}"
        
        if key in existing_keys:
            continue
            
        try:
            # Update progress bar description
            pbar.set_description(f"Processing {title[:20]}...+{persona}")
            
            system_prompt = create_ethical_system_prompt(persona)
            user_input = crete_evaluation_prompt(story)
            output = run(user_input, system_prompt)
            entire_output, answer = extract_answer(output)
            
            new_row = {
                'key': key,
                'title': title,
                'persona': persona,
                'entire_output': entire_output,
                'answer': answer,
                'target': target_label,
                'is_same': (answer == target_label),
                'full_output': output
            }
            
            df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
            existing_keys.add(key)
            df_results.to_csv(results_file, index=False)
            
            # Update progress bar
            pbar.update(1)
            
        except Exception as e:
            print(f"\nError processing {key}: {e}")
            pbar.update(1)  # Still update progress even on error
            continue

pbar.close()
print(f"\nCompleted! Final results saved to {results_file}")
print(f"Total processed: {len(df_results)}")

## Analysis of the results (First Approach)

In [ ]:
def filter_csv_by_titles(input_file="evaluation_results.csv", output_file="filtered_results.csv"):
    """
    Filter CSV rows where the 'title' column value is in the titles list
    """
    
    try:
        # Load the CSV file
        df = pd.read_csv(input_file)
        print(f"Loaded {len(df)} rows from {input_file}")
        
        # Check if title column exists
        if 'title' not in df.columns:
            print("Error: 'title' column not found in CSV")
            print(f"Available columns: {list(df.columns)}")
            return None
        
        # Filter rows where title is in the titles list
        filtered_df = df[df['title'].isin(titles)]
        
        print(f"Found {len(filtered_df)} rows matching titles from the list")
        print(f"Out of {len(titles)} titles in your list")
        
        # Save to new file
        filtered_df.to_csv(output_file, index=False)
        print(f"Filtered results saved to {output_file}")
        
        # Show some stats
        if len(filtered_df) > 0:
            unique_matches = filtered_df['title'].nunique()
            print(f"Matched {unique_matches} unique titles")
            
            # Show first few matches
            print("\nFirst few matching titles:")
            for title in filtered_df['title'].unique()[:5]:
                count = len(filtered_df[filtered_df['title'] == title])
                print(f"  - '{title}': {count} rows")
        
        return filtered_df
        
    except FileNotFoundError:
        print(f"Error: File '{input_file}' not found")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None


result_df = filter_csv_by_titles()   


In [ ]:
# Read the CSV file
df = pd.read_csv('evaluation_results.csv')

# Create the pivot table transformation
# Using 'title' as index, 'persona' as columns, and 'answer' as values
transformed_df = df.pivot_table(
    index='title',           # Each unique title becomes a row
    columns='persona',       # Each persona becomes a column
    values='answer',         # The values are the RIGHT/WRONG/NaN responses
    aggfunc='first'         # In case of duplicates, take the first value
).reset_index()

# Clean up the column names (remove the name from the columns index)
transformed_df.columns.name = None

# Add the target column (same for all 5 personas for each title)
# Get the target value for each title (should be the same across all personas)
target_mapping = df.groupby('title')['target'].first().to_dict()
transformed_df['target'] = transformed_df['title'].map(target_mapping)

# Reorder columns to have title first, then personas, then target
persona_order = ['conservative', 'progressive', 'libertarian', 'moderate', 'populist']
available_personas = [col for col in persona_order if col in transformed_df.columns]
column_order = ['title'] + available_personas + ['target']

# Select and reorder columns
transformed_df = transformed_df[column_order]

# Remove rows with any null values in persona columns
persona_columns = ['conservative', 'progressive', 'libertarian', 'moderate', 'populist']
transformed_df = transformed_df.dropna(subset=persona_columns)

# Remove rows where any persona column has values other than 'RIGHT' or 'WRONG'
valid_values = ['RIGHT', 'WRONG']
mask = transformed_df[persona_columns].isin(valid_values).all(axis=1)
transformed_df = transformed_df[mask]

# Reset the index to have clean sequential numbering
transformed_df = transformed_df.reset_index(drop=True)

# Display the result
print("Transformed DataFrame:")
print(f"Shape: {transformed_df.shape}")
print("\nFirst few rows:")
print(transformed_df.head())

# Optional: Save to CSV
# transformed_df.to_csv('transformed_persona_data.csv', index=False)

# Display summary statistics
print(f"\nSummary:")
print(f"Total unique titles: {len(transformed_df)}")
print(f"Personas: {available_personas}")
print(f"Response distribution across all personas:")
for persona in available_personas:
    if persona in transformed_df.columns:
        value_counts = transformed_df[persona].value_counts(dropna=False)
        print(f"  {persona}: {dict(value_counts)}")

In [ ]:
# Convert RIGHT/WRONG to numeric values for variance calculation
# RIGHT = 1, WRONG = 0
persona_columns = ['conservative', 'progressive', 'libertarian', 'moderate', 'populist']

# Create a copy for numeric conversion
df_numeric = transformed_df.copy()
for col in persona_columns:
    df_numeric[col] = df_numeric[col].map({'RIGHT': 1, 'WRONG': 0})

# Calculate variance for each row (across all personas)
df_numeric['variance'] = df_numeric[persona_columns].var(axis=1)

# Calculate mean agreement for each row (percentage of RIGHT responses)
df_numeric['mean_agreement'] = df_numeric[persona_columns].mean(axis=1)

# Calculate standard deviation as well
df_numeric['std_dev'] = df_numeric[persona_columns].std(axis=1)

print("Variance Analysis Summary:")
print(f"Mean variance across all scenarios: {df_numeric['variance'].mean():.4f}")
print(f"Standard deviation of variance: {df_numeric['variance'].std():.4f}")
print(f"Min variance: {df_numeric['variance'].min():.4f}")
print(f"Max variance: {df_numeric['variance'].max():.4f}")

print(f"\nScenarios with highest disagreement (variance > 0.2):")
high_variance = df_numeric[df_numeric['variance'] > 0.2]
if len(high_variance) > 0:
    for idx, row in high_variance.iterrows():
        print(f"  {row['title'][:60]}... (variance: {row['variance']:.3f})")
else:
    print("  No scenarios with variance > 0.2")

# Create scatter plot
plt.figure(figsize=(12, 8))

# Scatter plot: mean agreement vs variance
scatter = plt.scatter(df_numeric['mean_agreement'], df_numeric['variance'], 
                     alpha=0.7, s=60, c=df_numeric.index, cmap='viridis')

plt.xlabel('Mean Agreement (Proportion of RIGHT responses)', fontsize=12)
plt.ylabel('Variance across Personas', fontsize=12)
plt.title('Political Persona Agreement vs Disagreement in Ethical Judgments', fontsize=14)

# Add grid for better readability
plt.grid(True, alpha=0.3)

# Add colorbar
plt.colorbar(scatter, label='Scenario Index')

# Annotate extreme points
# Most agreement (lowest variance)
min_var_idx = df_numeric['variance'].idxmin()
plt.annotate(f'Most Agreement\n{df_numeric.loc[min_var_idx, "title"][:30]}...', 
             xy=(df_numeric.loc[min_var_idx, 'mean_agreement'], 
                 df_numeric.loc[min_var_idx, 'variance']),
             xytext=(10, 10), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgreen', alpha=0.7),
             fontsize=8)

# Most disagreement (highest variance)
max_var_idx = df_numeric['variance'].idxmax()
plt.annotate(f'Most Disagreement\n{df_numeric.loc[max_var_idx, "title"][:30]}...', 
             xy=(df_numeric.loc[max_var_idx, 'mean_agreement'], 
                 df_numeric.loc[max_var_idx, 'variance']),
             xytext=(10, -30), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', alpha=0.7),
             fontsize=8)

plt.tight_layout()
plt.show()

# Create a detailed analysis DataFrame
analysis_df = df_numeric[['title', 'conservative', 'progressive', 'libertarian', 
                         'moderate', 'populist', 'variance', 'mean_agreement', 'std_dev']].copy()

# Sort by variance (highest disagreement first)
analysis_df = analysis_df.sort_values('variance', ascending=False)

print(f"\nTop 5 Most Controversial Scenarios (highest variance):")
print(analysis_df.head()[['title', 'variance', 'mean_agreement']].to_string(index=False))

print(f"\nTop 5 Most Agreed Upon Scenarios (lowest variance):")
print(analysis_df.tail()[['title', 'variance', 'mean_agreement']].to_string(index=False))

# Additional visualization: Heatmap of persona responses
plt.figure(figsize=(10, 8))
persona_responses = df_numeric[persona_columns + ['title']].set_index('title')
sns.heatmap(persona_responses.T, 
            cmap='RdYlGn', 
            cbar_kws={'label': 'Response (0=WRONG, 1=RIGHT)'},
            xticklabels=[title[:30] + '...' for title in persona_responses.index],
            yticklabels=['Conservative', 'Progressive', 'Libertarian', 'Moderate', 'Populist'])
plt.title('Persona Response Patterns Across All Scenarios')
plt.xlabel('Scenarios')
plt.ylabel('Political Personas')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Return the analysis dataframe
print(f"\nFinal analysis DataFrame shape: {analysis_df.shape}")
analysis_df